# OpenAI API and nice-to-have wrappers

:)

In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Vanilla API: Synchronous Version

In [50]:
from openai import OpenAI
# API call parameters
from openai.types.chat.chat_completion_message_param import (
    ChatCompletionMessageParam, # i.e. any message
    ChatCompletionSystemMessageParam, # system message
    ChatCompletionUserMessageParam, # user message
    ChatCompletionAssistantMessageParam, # assistant message (generated by model)
    ChatCompletionToolMessageParam, # result of tool call
)
# API return values
from openai.types.chat import (
    ChatCompletion, # Overall Completion, has id, stats, choices
    ChatCompletionMessage, # completion.choice[0], has role, content, tool_calls
)
from openai.types.chat.chat_completion_message_tool_call import (
    ChatCompletionMessageToolCall, # a tool call with id, function and type (only function is supported)
    Function, # function call with name and arguments
)

client = OpenAI()

In [4]:
kwargs = dict(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo-1106",
)

def handle_function_call(call: Function) -> str:
    print(f"calling {call.name} with {call.arguments}")
    # lookup function, parse args, call them, then return result
    return 'success'

def handle_completion(completion: ChatCompletion) -> None | [ChatCompletionToolMessageParam]:
    print(completion)

    # Whatever we want to actually do with the response
    # handle message
    message = completion.choices[0].message

    # handle tool calls
    calls = message.tool_calls
    if calls:
        results: list[ChatCompletionToolMessageParam] = []
        for call in calls:
            if call.type == "function":
                result = handle_function_call(call.function)
                # Note that here, we would usually append the result to the message list for the next call, instead return
                results.append(ChatCompletionToolMessageParam(
                    tool_call_id=call.id, # for parallel calls
                    role='tool',
                    content=result,
                ))
        return results # tool call results
    else:
        return None # nothing was called

MAX_CALLS = 10

TypeError: unsupported operand type(s) for |: 'NoneType' and 'list'

### Non-Streaming

As Vanila as it gets:

In [31]:
completion: ChatCompletion = client.chat.completions.create(**kwargs)
# do something with completion, i.e. messages, tool calls
handle_completion(completion)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletion(id='chatcmpl-8NK7UCIJUuuGfqlNWPyauxRRm6Ffr', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='This is a test', role='assistant', function_call=None, tool_calls=None))], created=1700569576, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_eeff13170a', usage=CompletionUsage(completion_tokens=4, prompt_tokens=12, total_tokens=16))


Converting the PyDantic models to dict:

In [33]:
completion.model_dump()

{'id': 'chatcmpl-8NK7UCIJUuuGfqlNWPyauxRRm6Ffr',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': 'This is a test',
    'role': 'assistant',
    'function_call': None,
    'tool_calls': None}}],
 'created': 1700569576,
 'model': 'gpt-3.5-turbo-1106',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_eeff13170a',
 'usage': {'completion_tokens': 4, 'prompt_tokens': 12, 'total_tokens': 16}}

Simple back-and-forth conversion:

In [42]:
kwargs_without_messages = kwargs.copy()
del kwargs_without_messages['messages']

messages = []

while True:
    message = input("You: ")
    if message == 'exit':
        break
    messages.append(ChatCompletionUserMessageParam(role='user', content=message))

    completion: ChatCompletion = client.chat.completions.create(
        messages=messages,
        **kwargs_without_messages,
    )
    response: ChatCompletionMessage = completion.choices[0].message
    print(f"ChatGPT: {response.content}")
    messages.append(response)
    print(messages)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatGPT: Hi!
[{'role': 'user', 'content': 'say "hi"'}, ChatCompletionMessage(content='Hi!', role='assistant', function_call=None, tool_calls=None)]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatGPT: Hi! Hi!
[{'role': 'user', 'content': 'say "hi"'}, ChatCompletionMessage(content='Hi!', role='assistant', function_call=None, tool_calls=None), {'role': 'user', 'content': 'again, but twice!'}, ChatCompletionMessage(content='Hi! Hi!', role='assistant', function_call=None, tool_calls=None)]


In [45]:
[type(m) for m in messages]

[dict,
 openai.types.chat.chat_completion_message.ChatCompletionMessage,
 dict,
 openai.types.chat.chat_completion_message.ChatCompletionMessage]

In [46]:
for m in messages:
    print(m['content'])

say "hi"


TypeError: 'ChatCompletionMessage' object is not subscriptable

As you can see, since the API returns a Pydantic Model, we need to convert it into a dict before appending it to the `messages`` list (which gets fed back into the API). The API does deal with it gracefully, but it is confusing for us from a user perspective.

### Tool calls, implemented in a simple for loop:

It's not that trivial to define the function schema, so see below for much easier auto-convert options

In [ ]:
tools = []

In [ ]:
kwargs_without_msg = kwargs.copy()
messages = kwargs_without_msg.pop('messages')

for i in range(MAX_CALLS):
    completion: ChatCompletion = client.chat.completions.create(
        messages=messages,
        **kwargs_without_msg
    )
    # do something with completion, i.e. messages, tool calls
    tool_call_results = handle_completion()

    if tool_call_results:
        messages.append(tool_call_results)
    else:
        # IMPORTANT no more tool calls, we're done
        break


In [ ]:

# def generate_tool_calls(**kwargs, max_calls=MAX_CALLS):
#     for i in range(max_calls):
#         completion = client.chat.completions.create(**kwargs)
#         yield completion
#         if completion contains no tool calls:
#             break
            
# completions: Generator[ChatCompletion] = generate_tool_calls(client, **kwargs)
# for completion in completions:
#     # do something with completion, i.e. messages, tool calls
#     handle_completion()


# [streaming, generator that yields partial completions]


## Nice-To-Haves

## Message History Management
While not the hardest thing to manage, keeping a `messages` list object directly can lead to some annoyances and bugs, because:
1. You need to always do in-place operations to make sure the reference to the original list is kept.
2. The return value of API is a PyDantic model ChatCompletionMessage, which is not a dict, leading to inconsistent types.
3. When managing long conversation history, you might want to start truncating really old messages or filter out irrelevant parts (using sementic similarity, for example) to stay under context size limitations and also save tokens.
4. When using RAG, you could define the user-prompt preprocessing step here.
5. When using more complicated system message scheme that keeps track of global state (a trick I like to use in copilots), you want to be able to define the system message creator.

In [5]:
from gpt_wrapper.messages import MessageHistory

In [6]:
messages = MessageHistory()
messages.add_user("Hello")
messages.add_assistant({'role': 'AI', 'message': "Hi"})

### Token Useage Tracker

In [7]:
from gpt_wrapper.trackers import ChatCompletionUsageTracker

### Toolkit Class

For autoconverting, validating, and also for any shared states for a group of functions.

In [25]:
from pydantic import BaseModel, Field
from gpt_wrapper.tools import FunctionTool, Toolkit

In [26]:
# Example tool
class Location(BaseModel):
    '''Location to get weather for'''
    city: str
    state: str

class get_weather(FunctionTool):
    '''return temp in Fahrenheit'''
    location: Location = Field(..., description="Location to get weather for")

    def __call__(args, state=None):
        # call weather api
        return f'Weather at {args.location} is 72F' if args.location else 'Error: no location provided'

get_weather.to_openai()

{'type': 'function',
 'function': {'name': 'get_weather',
  'description': 'return temp in Fahrenheit',
  'parameters': {'type': 'object',
   'properties': {'location': {'description': 'Location to get weather for',
     'properties': {'city': {'type': 'string'}, 'state': {'type': 'string'}},
     'required': ['city', 'state'],
     'type': 'object'}},
   'required': ['location']}}}

In [57]:
class NotepadState(BaseModel):
    content: str = ''

class NotepadToolkit(Toolkit):
    def __init__(self):
        super().__init__(tools=[
            self.read,
            self.write,
        ])
        self.state: NotepadState = NotepadState()
    
    class read(FunctionTool):
        '''read the notepad'''
        def __call__(args, state: NotepadState):
            return state.content
    
    class write(FunctionTool):
        '''write to the notepad'''
        text: str = Field(..., description="text to write to notepad")
        async def __call__(args, state: NotepadState):
            state.content += args.text
            return "success!"

toolkit = NotepadToolkit()
toolkit.to_openai()

[{'type': 'function',
  'function': {'name': 'read',
   'description': 'read the notepad',
   'parameters': {'type': 'object', 'properties': {}, 'required': []}}},
 {'type': 'function',
  'function': {'name': 'write',
   'description': 'write to the notepad',
   'parameters': {'type': 'object',
    'properties': {'text': {'description': 'text to write to notepad',
      'type': 'string'}},
    'required': ['text']}}}]

In [41]:
lookup = toolkit.to_tool_lookup()
w = lookup['write']
await w(text='hello')

'New content: hello'

as you can see, async or not, it can be handled the same way:

In [44]:
r = lookup['read']
r()

'hello'

### Tool Calling with Auto-Conversion

In [58]:
from gpt_wrapper.assistant import call_requested_function

In [60]:
notepad = NotepadToolkit()
notepad.state.content = "Surprise! There was already something written here!\n"
tools = notepad.to_openai()
lookup = notepad.to_tool_lookup()

messages = [
    {"role": "user", "content": "Write hi to the notepad, and the read the whole content back to me, verbatim!"},
]

for i in range(10):
    completion: ChatCompletion = client.chat.completions.create(
        messages=messages,
        tools=tools,
        model="gpt-3.5-turbo-1106",
    )
    
    response: ChatCompletionMessage = completion.choices[0].message
    messages.append(response)

    if response.content:
        # handle message
        print(f"ChatGPT: {response.content}")
    if response.tool_calls:
        for call in response.tool_calls:
            # handle tool calls (usually nothing more than logging/streaming)
            print(f"Call Request: {call.function}")
            result = await call_requested_function(call.function, lookup)
            print(f"Tool Result: {result}")
            messages.append(ChatCompletionToolMessageParam(
                tool_call_id=call.id, # for parallel calls
                role='tool',
                content=result,
            ))
    else:
        # IMPORTANT no more tool calls, we're done
        break


Call Request: Function(arguments='{"text": "hi"}', name='write')
awaiting coroutine
Tool Result: success!
Call Request: Function(arguments='{}', name='read')
Tool Result: Surprise! There was already something written here!
hi
ChatGPT: The content of the notepad is "Surprise! There was already something written here! hi".
